In [9]:
from utils import *

In [10]:
import numpy as np
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from kaggle_environments import evaluate, make
import kaggle_environments


In [11]:
env = ConnectX()
gamma = 0.99
copy_step = 25
hidden_units = [128, 128, 128, 128, 128]
max_experiences = 10000
min_experiences = 100
batch_size = 32
lr = 1e-2
epsilon = 0.5
decay = 0.999
min_epsilon = 0.01
episodes = 50000
precision = 7
# import pdb; pdb.set_trace()
# prepare the agents

num_states = env.observation_space.n
num_actions = env.action_space.n

all_total_rewards = np.empty(episodes)
all_avg_rewards = np.empty(episodes) # Last 100 steps
all_epsilons = np.empty(episodes)

# Initialize models
TrainNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
TargetNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)


In [33]:
for layer in list(TrainNet.model.children()):
    print(layer.)
    
# We have to implement forward() without pytorch




Conv2d(1, 16, kernel_size=(2, 2), stride=(1, 1))
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1))
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Linear(in_features=384, out_features=7, bias=True)


Linear(in_features=384, out_features=7, bias=True)

In [34]:
conv1 = list(TrainNet.model.children())[0]

In [46]:
import json
f"{conv1.state_dict()['weight'].numpy()}"
f"{conv1.state_dict()['bias'].numpy()}"

'[  3.318561     7.9553237    7.576386    -4.5332494   10.648888\n  -3.7517543    0.23433919   5.081022     7.467227    -2.4467227\n  -6.4362335    0.150005   -10.304609   -11.994086     1.409211\n   3.5867472 ]'

In [20]:
fc_layers = []

# Get all hidden layers' weights
for i in range(len(hidden_units)):
    fc_layers.extend([
        TrainNet.model.hidden_layers[i].weight.T.tolist(), # weights
        TrainNet.model.hidden_layers[i].bias.tolist() # bias
    ])

# Get output layer's weights
fc_layers.extend([
    TrainNet.model.output_layer.weight.T.tolist(), # weights
    TrainNet.model.output_layer.bias.tolist() # bias
])

# Convert all layers into usable form before integrating to final agent
fc_layers = list(map(
    lambda x: str(list(np.round(x, precision))) \
        .replace('array(', '').replace(')', '') \
        .replace(' ', '') \
        .replace('\n', ''),
    fc_layers
))
fc_layers = np.reshape(fc_layers, (-1, 2))

# Create the agent
my_agent = '''def my_agent(observation, configuration):
    import numpy as np

'''

# Write hidden layers
for i, (w, b) in enumerate(fc_layers[:-1]):
    my_agent += '    hl{}_w = np.array({}, dtype=np.float32)\n'.format(i+1, w)
    my_agent += '    hl{}_b = np.array({}, dtype=np.float32)\n'.format(i+1, b)
# Write output layer
my_agent += '    ol_w = np.array({}, dtype=np.float32)\n'.format(fc_layers[-1][0])
my_agent += '    ol_b = np.array({}, dtype=np.float32)\n'.format(fc_layers[-1][1])

my_agent += '''
    state = observation.board[:]
    state.append(observation.mark)
    out = np.array(state, dtype=np.float32)

'''

# Calculate hidden layers
for i in range(len(fc_layers[:-1])):
    my_agent += '    out = np.matmul(out, hl{0}_w) + hl{0}_b\n'.format(i+1)
    my_agent += '    out = 1/(1 + np.exp(-out))\n' # Sigmoid function
# Calculate output layer
my_agent += '    out = np.matmul(out, ol_w) + ol_b\n'

my_agent += '''
    for i in range(configuration.columns):
        if observation.board[i] != 0:
            out[i] = -1e7

    return int(np.argmax(out))
    '''

AttributeError: 'CNN_model' object has no attribute 'hidden_layers'